<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ejercicio de clasificación con redes neuronales convolucionales (CNN)

Ejemplo de clasificación utilizando redes neuronales convolucionales para la clasificación de imagenes<br>

v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import keras
from keras.models import Sequential
from keras.utils import to_categorical

from glob import glob
import matplotlib.image as mpimg

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

### `Simpsons dataset`:
El dataset **`Simpsons`** contiene 550Mbytes de imagenes a color de los personajes de los Simpsons (47 personajes). Cada imagen es de tiene al rededor de 500x450 píxeles a color (3 canales).<br> [Dataset source](https://www.kaggle.com/paultimothymooney/zipfiles)

In [ ]:
from google.colab import drive
# Como el dataset es muy pesado ya se descargó proviamente del link del dataset
# se y subió a una carpeta de nuestro google drive
# Conectamos el colab con nuestra carpeta d google drive
drive.mount("/content/drive")

In [ ]:
# Nos vemos a la carpeta en donde está la carpeta simpsons_dataset
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/simpsons")

In [ ]:
# Si la carpeta se encuentra zipiada ejecutar
# Esta operación puede tomar un rato
# Descargar datos de test
if os.access('simpsons_dataset', os.F_OK) is False:
    !unzip -q simpsons_dataset.zip
else:
    print("El archivo ya se encuentra descargado")

In [ ]:
# Visualizar los directiorios o tipos de personas
os.listdir("./simpsons_dataset")

In [ ]:
personajes = os.listdir("./simpsons_dataset")
print("Cantidad de tipos de personaejs:", len(personajes))

In [ ]:
# Visualizar las 10 primeras imagenes de un personaje
files = glob("./simpsons_dataset/" + personajes[0] + "/**.jpg")

fig = plt.figure(figsize=(16,9))
for i in range(10):
    ax = fig.add_subplot(2, 5, i+1)
    ax.axis('off')
    img = mpimg.imread(files[i])
    plt.imshow(img)
plt.show()

In [ ]:
# Visualizar la dimension de la primera imagen
img = mpimg.imread(files[0])
img.shape

In [ ]:
# Visualizar como están representados los pixeles
print(img[85, 100:110, :])

#### Conclusiones
- Las imagenes tienen tamaño variable, utilizaremos un tamaño reducido para que todas las imagenes sean iguales (se elije 150x150)
- Las imagenes están representadas de 0 a 255, hay que normalizarlas

In [ ]:
# Analizar cuantos personajes hay de cada uno
nombre_personajes = []
cantidad_personajes = []
for personaje in personajes:
    nombre_personaje = personaje.split("_")[0]
    files = glob("./simpsons_dataset/" + personajes[0] + "/**.jpg")
    nombre_personajes.append(nombre_personaje)
    cantidad_personajes.append(len(files))

print("Cantidad de", nombre_personajes[0], ":", cantidad_personajes[0])
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot()
sns.barplot(x=nombre_personajes, y=cantidad_personajes, ax=ax)
plt.show()

Se puede ver que el dataset está balanceado

In [ ]:
# Descargar datos de test
if os.access('simpsons_test', os.F_OK) is False:
    if os.access('simpsons_test.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://github.com/InoveAlumnos/dataset_analytics_python/raw/master/simpsons_test.zip > simpsons_test.zip
        else:
            !wget simpsons_test.zip https://github.com/InoveAlumnos/dataset_analytics_python/raw/master/simpsons_test.zip
    !unzip -q simpsons_test.zip
else:
    print("El archivo ya se encuentra descargado")

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Crear un generador, indicando si deseamos realizar un escalado de la imagen
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        directory="./simpsons_dataset",
        target_size=(150, 150),
        batch_size=20,
        class_mode="categorical")

index_to_classes = dict(zip(train_generator.class_indices.values(), train_generator.class_indices.keys()))


# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# El generador "train_generator" se lo puede utilizar para acceder a los datos
# de a cantidad batch de imagenes. En este caso el generador me retornará
# la primera vez las primeras 20 imagenes
# El generador devuelve las imagenes (X) y las clases(personaes) a las que
# pertenece (y)
# X, y = train_generator.next()
batch_imagenes, batch_clases = train_generator.next()

In [ ]:
batch_imagenes.shape

In [ ]:
batch_clases.shape

In [ ]:
print("Cantidad de imagenes en el batch:", batch_imagenes.shape[0])
print("Dimensión de la imagen:", batch_imagenes.shape[1:])

In [ ]:
print("Cantidad de clases/personajes:", batch_clases.shape[1])

In [ ]:
# Observar las primeras 5 imagenes de ese batch
fig = plt.figure(figsize=(16,9))
for i in range(5):
    ax = fig.add_subplot(1, 5, i+1)
    ax.imshow(batch_imagenes[i])
    numero_clase = batch_clases[i].argmax()
    ax.set_title(index_to_classes[numero_clase])
plt.show()

__Importante__! Luego de usar un generador "jugando", ese batch de imagenes que sacamos ya no se encontrará disponible para ser utilizado en el entrenamiento, es recomendable volver a crear los generadores si se los consumen

In [ ]:
# Crear un generador, indicando si deseamos realizar un escalado de la imagen
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        directory="./simpsons_dataset",
        target_size=(150, 150),
        batch_size=20,
        class_mode="categorical")

index_to_classes = dict(zip(train_generator.class_indices.values(), train_generator.class_indices.keys()))

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

In [ ]:
# Los generadores ya que encargan de transformar la salida a oneHotEncoding

In [ ]:
# input shape (observado del análisis de datos)
in_shape = (150, 150, 3)
in_shape

In [ ]:
# output shape (observado del análisis de datos)
out_shape = 47
out_shape

In [ ]:
# Debemos definir cuantas imagenes se consumiran por epoca (steps_per_epoch)
# ya que estando el generador en el medio Keras no puede saberlo por
# su cuenta
steps_per_epoch_train = len(train_generator)
steps_per_epoch_train

In [ ]:
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

model1 = Sequential()

# Primero realizaremos un modelo muy simple con una solo par de CONV + POOL
# tal cual se utilizo en los otros notebooks de dataset más simples

model1.add(Conv2D(filters=8, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=in_shape))
model1.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model1.add(Flatten())
model1.add(Dense(units=64, activation='relu'))
model1.add(Dense(units=out_shape, activation='softmax'))

model1.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Se puede observar que esta red tiene más de 2 millones de parámetros para entrenar!!\
Esto es porque la capa densa de POOL de 75x75x8 se transforma a un flatten
de 450000 neuronaes (75x75x8 = 45000) que luego se conectan con todas las 
neuroanes de la capa sigueinte (64) --> 45000x64 + 64 = 2880064\
Para bajar la cantidad de parametros debemos seguir comprimiendo la imagen


In [ ]:
history1 = model1.fit(train_generator, steps_per_epoch=steps_per_epoch_train, epochs=2)

In [ ]:
epoch_count = range(1, len(history1.history['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=history1.history['accuracy'], label='train')
plt.show()

In [ ]:
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

model2 = Sequential()

# Ahora agregaremos más pares de capas CONV + POOL a fin de reducir más la
# dimensión de la imagen antes de llegar a la capa flatten
# Otra estrategia es ir aumentando la cantidad de filtros a medida que crece
# la profundidad de la red

# convolucional f=(3,3), # de filtros: 8, activación relu
# max pooling f=2, s=2
model2.add(Conv2D(filters = 8, kernel_size = (3, 3), strides=1, padding='same', activation='relu', input_shape=(150, 150, 3)))
model2.add(MaxPooling2D(pool_size=2, strides=2, padding='valid'))
# convolucional f=(3,3), # de filtros: 16, activación relu
# max pooling f=2, s=2
model2.add(Conv2D(filters = 16, kernel_size = (3, 3), strides=1, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2, strides=2))
# convolucional f=(3,3), # de filtros: 32, activación relu
# max pooling f=2, s=2
model2.add(Conv2D(filters = 32, kernel_size = (3, 3), strides=1, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2, strides=2))
# convolucional f=(3,3), # de filtros: 64, activación relu
# max pooling f=2, s=2
model2.add(Conv2D(filters = 64, kernel_size = (3, 3), strides=1, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2, strides=2))
# capa flatten
model2.add(Flatten())
# capa densa de 64 elementos activación relu
model2.add(Dense(units=128, activation='relu'))
model2.add(Dropout(rate=0.2))
# capa densa con un output de 10 elemento con activación softmax
model2.add(Dense(units=out_shape, activation='softmax'))

model2.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model2.summary()

In [ ]:
history2 = model2.fit(train_generator, steps_per_epoch=steps_per_epoch_train, epochs=2)

In [ ]:
epoch_count = range(1, len(history2.history['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=history2.history['accuracy'], label='train')
plt.show()

In [ ]:
# Predecir los datos
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        directory="./simpsons_test",
        target_size=(150, 150),
        batch_size=10,
        class_mode=None,
        shuffle=False)

y_hat_prob = model2.predict(test_generator)
y_hat_prob[0]

In [ ]:
y_hat = np.argmax(y_hat_prob,axis=1)
y_hat

In [ ]:
#¿Cómo obtenemos el "y" verdadero?
test_generator.filenames

In [ ]:
# Muy rebuscada esta forma de obtener los nombres de los personajes!
# Pero en general cuando tenemos los datos de test no tenemos los nombres
# por lo que no tenemos el "y" verdadero
personajes_test = []
for file in test_generator.filenames:
    image_name = os.path.basename(file)
    image_name_split = image_name.split("_")
    personaje_name_split = image_name_split[:len(image_name_split)-1]
    personaje = personaje_name_split[0]
    for name in personaje_name_split[1:]:
        personaje += "_" + name
    personajes_test.append(personaje)
personajes_test

In [ ]:
# Obtener el "y" verdadero
y_test = [train_generator.class_indices[personaje] for personaje in personajes_test]
y_test

In [ ]:
# Descargar el modelo entrenado para usar en el futuro sin tener
# que volver a entrenarlo
model2.save("cnn_simpsons.h5")

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_hat)
cmd = ConfusionMatrixDisplay(cm, display_labels=range(47))
cmd.plot(cmap=plt.cm.Blues)
plt.show()

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

Se utiliza el ranking de los peores 10 números clasificados con una ANN para evlauar contra este nuevo modelo de red neuronal

In [ ]:
batch_test = test_generator.next()

In [ ]:
# Observar las primeras 5 imagenes de ese batch
fig = plt.figure(figsize=(16,9))
for i in range(10):
    ax = fig.add_subplot(2, 5, i+1)
    ax.imshow(batch_test[i])
    numero_clase = y_hat[i]
    ax.set_title(index_to_classes[numero_clase])
plt.show()

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

Al utilizar más pares de capas CONV+POOL se pudo obtener un mejor resultado, un modelo casi perfecto. Hay que tener en cuenta que el dataset de test es muy pequeño y hay muchos otros personajes que no estamos clasificando.